In [1]:
import os

os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

import pandas as pd
import gc

from bokeh.io import output_notebook, show
output_notebook()

from src.dao import csv_dao
from src.entity.stop_region import StopRegion, StopRegionGroup, sr_row_to_stop_region
from src.entity.stop_region import group_stop_regions_for_agglutination, same_closest_poi
from src.poi_grabber import google_places, hot_osm
from src.plot import plot
from src.utils import geo

from src.plot.plot2 import add_hot_osm_pois


# pd.set_option('display.float_format', lambda x: '%.4f' % x)

# import warnings
# warnings.filterwarnings('ignore')

# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

working dir /home/tales/dev/master/mdc_analysis


Loading BokehJS ...

In [2]:
def plot_sr_group(user_id):
    print("Loading data")
    sr_list = csv_dao.stop_region_sequence(user_id)
    print("Agglutinating consecutive stop regions")
    stop_regions_group = StopRegionGroup(sr_list).agglutinate_stop_regions()
    print("Stop Regions Group size: {}".format(stop_regions_group.size()))
    
    p = stop_regions_group.plot(fill_color="magenta", plot_n_pois=6, title="User: {}".format(user_id))
    p = mark_home_and_work(p, stop_regions_group)
    
    return p, stop_regions_group
    
def mark_home_and_work(plot, sr_group):
    for sr in sr_group.stop_region_list:
        if "HOME" in sr.semantics:
            plot = sr.plot_simple(p=plot, color="lightblue", legend="HOME")

        if "WORK" in sr.semantics:
            plot = sr.plot_simple(p=plot, color="lightgreen", legend="WORK")
    return plot    

## USER 6070

In [3]:
plot, stop_regions_group = plot_sr_group(6070)

Loading data
Agglutinating consecutive stop regions
Stop Regions Group size: 161


In [4]:
hot_osm_pois = hot_osm.load_hot_osm_pois(valid_pois=True)
hot_osm_pois["latitude"] = hot_osm_pois["lat_4326"]
hot_osm_pois["longitude"] = hot_osm_pois["lon_4326"]

/home/tales/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (3,4,6,8,9,11,13,15,16,19,20,24,25,27,28,29,32,33,34,35,37,39,40,41,43,47,50,52,53,54,56,57,59,60,63,64,65,66,69) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [5]:
def get_hot_osm_pois(sr_region_cases, hot_osm_pois):
    osm_close = pd.DataFrame()

    for case_sr in sr_region_cases:
        osm_close = osm_close.append(geo.slice_geo_data2(hot_osm_pois, case_sr.centroid_lat, case_sr.centroid_lon, search_tolerance=0.005))
        
    print("Total OSM HOT POIs: {}".format(len(osm_close)))
    
    return osm_close

In [6]:
case_sr = stop_regions_group.search_stop_region_by_id("agg_6070_133")
osm_close = get_hot_osm_pois([case_sr], hot_osm_pois)

Total OSM HOT POIs: 20


In [7]:
p = add_hot_osm_pois(pois=osm_close, size=8, p=plot)

Look at Stop Regions: <b>agg_6070_133</b> and <b>agg_6070_136</b>

Esse plot são os Stop Regions com os 5 pois mais pertos

In [8]:
show(p)

## Another Users

In [9]:
user_id = pd.Series(csv_dao.list_stop_region_usernames()).sample().item()
print(user_id)

plot, stop_regions_group = plot_sr_group(user_id)
show(plot)

6179
Loading data
Agglutinating consecutive stop regions
Stop Regions Group size: 158


In [10]:
user_id = pd.Series(csv_dao.list_stop_region_usernames()).sample().item()
print(user_id)

plot, stop_regions_group = plot_sr_group(user_id)
show(plot)

6027
Loading data
Agglutinating consecutive stop regions
Stop Regions Group size: 251


In [11]:
user_id = pd.Series(csv_dao.list_stop_region_usernames()).sample().item()
print(user_id)

plot, stop_regions_group = plot_sr_group(user_id)
show(plot)

6005
Loading data
Agglutinating consecutive stop regions
Stop Regions Group size: 249
